In [1]:
from Agent_Tree_5 import *

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Hyperparameters
gamma = 0.75
nb_trials = 100
nb_episodes = 10000
learning_rate = 7e-6
eps = 1e-7
beta_v = 0.05
beta_e = 0.05
nb_agent = 5

# Prepare

In [3]:
rail_generator = sparse_rail_generator(max_num_cities=10)

env = RailEnv(
    width=30, 
    height=30, 
    number_of_agents=nb_agent,
    rail_generator=rail_generator,
    line_generator=sparse_line_generator(),
    obs_builder_object=TreeObsForRailEnv(max_depth=2, predictor=ShortestPathPredictorForRailEnv(30)),
    )

# env.stop_penalty = -1
# env.invalid_action_penalty = -1
# env.alpha = 1

In [5]:
obs, _ = env.reset()
obs = normalize_observation(obs[1], tree_depth=2, observation_radius=10)
print(obs.shape)

(231,)


/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 3/4
  warnings.warn(city_warning)


In [5]:
# Save Paths
path = "train_LSTMCell_A2C/" + datetime.now().strftime("%m%d-%H:%M:%S") + "_adam_lr7e-6_g0.75_tree_noEntropy"
log_dir = path+'/logs/'
ckpt_dir = path+'/ckpt/'
train_summary_writer = tf.summary.create_file_writer(log_dir)

2023-01-16 11:00:41.803415: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
agent_id = 0
agent = LSTMAgent(learning_rate, gamma, beta_v, beta_e,  #loss func
                 env, nb_trials, nb_agent, nb_episodes,path,
                 nb_hidden = 128)

# Train

In [7]:
agent.train()

/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 2/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 1/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:262: UserWarning: [WARNING] Changing to Grid mode to place at least 2 cities.
  warnings.warn("[WARNING] Changing to Grid mode to place at least 2 cities.")


KeyboardInterrupt: 

In [8]:
agent.model.save(path+'/model.h5')

# Test

In [28]:
test_model = keras.models.load_model("train_LSTMCell_A2C/0116-10:48:29_adam_lr7e-4_g0.9_tree/model.h5")
#test_model = agent.model

In [29]:

def run_episode(env, model):
    next_obs, _ = env.reset()
    score = 0
    reward = 0.0
    action_onehot = np.zeros((agent.nb_actions))
    cell_state = [tf.zeros((1,agent.nb_hidden)),tf.zeros((1,agent.nb_hidden))]
        
    for timestep in range(100):
        input = agent.prepare_input(next_obs, agent_id, action_onehot,reward,timestep)
        
        action_probs, critic_value, cell_state = model([input,cell_state[0],cell_state[1]])
        
        action_probs = tf.squeeze(action_probs)
        action = np.random.choice(agent.nb_actions, p=action_probs.numpy())
        action_onehot = np.zeros((agent.nb_actions))
        action_onehot[action] = 1

        next_obs, all_rewards, dones, info = env.step({agent_id: action})

        for agent_handle in env.get_agent_handles():
            score += all_rewards[agent_handle]

        #render_env(env)
        #print('Timestep {}, action = {}, total score = {}'.format(timestep, action_map[action], score))
        #tf.print(action_probs)

        if dones['__all__']:
            #print('All done!')
            # print(info['state'][0])
            # print(env.agents[0].latest_arrival, env._max_episode_steps)
            #print(env.agents[agent_id].position, env.agents[agent_id].target)

            if info['state'][0] == TrainState.DONE:
                #render_env(env)
                print('Timestep {}, action = {}, total score = {}'.format(timestep, action_map[action], score))
            return score

    print("Episode didn't finish after 100 timesteps.")
    return score

In [30]:
# print(env.agents[0].earliest_departure)
# print(env.agents[0].latest_arrival)

4
75


In [31]:
out = []
for i in range(100):
    out.append(run_episode(env, test_model))

    
print(out)
print(out.count(0)/len(out))

TrainState.MOVING
52 57
TrainState.DONE
24 26
Timestep 15, action = left, total score = 0


/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 3/4
  warnings.warn(city_warning)


TrainState.MOVING
83 88
Episode didn't finish after 100 timesteps.
TrainState.MOVING
28 30
TrainState.DONE
42 47
Timestep 28, action = left, total score = 0
TrainState.MOVING
45 49
TrainState.MOVING
40 43
TrainState.DONE
121 132
Timestep 85, action = left, total score = 0
TrainState.DONE
47 51
Timestep 31, action = left, total score = 0
TrainState.MOVING
21 23
TrainState.MOVING
62 66
TrainState.DONE
92 102
Timestep 82, action = left, total score = 0
Episode didn't finish after 100 timesteps.
TrainState.DONE
29 32
Timestep 19, action = left, total score = 0
TrainState.DONE
63 71
Timestep 62, action = left, total score = 0
TrainState.MOVING
38 43
TrainState.MOVING
49 54
Episode didn't finish after 100 timesteps.
TrainState.MOVING
49 54
TrainState.MOVING
71 80
Episode didn't finish after 100 timesteps.
TrainState.DONE
51 54
Timestep 34, action = left, total score = 0
TrainState.MOVING
54 57
TrainState.MOVING
51 57


/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 1/4
  warnings.warn(city_warning)
/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:262: UserWarning: [WARNING] Changing to Grid mode to place at least 2 cities.
  warnings.warn("[WARNING] Changing to Grid mode to place at least 2 cities.")


TrainState.MOVING
73 80
TrainState.MOVING
21 23
TrainState.MOVING
48 53
TrainState.MOVING
24 26


/Users/Marvin/opt/miniconda3/lib/python3.9/site-packages/flatland/envs/rail_generators.py:350: UserWarning: Could not set all required cities! Created 2/4
  warnings.warn(city_warning)


TrainState.MOVING
60 64
TrainState.MOVING
66 74
TrainState.DONE
63 68
Timestep 44, action = left, total score = 0
TrainState.MOVING
46 49
TrainState.MOVING
69 78
TrainState.MOVING
33 36
TrainState.MOVING
64 71
TrainState.DONE
57 64
Timestep 40, action = left, total score = 0
TrainState.DONE
86 97
Timestep 59, action = left, total score = 0
TrainState.DONE
34 37
Timestep 22, action = left, total score = 0
TrainState.MOVING
61 66
TrainState.MOVING
48 51
TrainState.DONE
26 29
Timestep 19, action = left, total score = 0
TrainState.MOVING
57 64
TrainState.MOVING
69 73
TrainState.MOVING
28 30
TrainState.MOVING
92 98
TrainState.MOVING
71 78
TrainState.DONE
25 27
Timestep 19, action = left, total score = 0
TrainState.MOVING
30 32
TrainState.MOVING
53 57
TrainState.MOVING
75 80
TrainState.MOVING
42 47
TrainState.MOVING
39 43
TrainState.DONE
72 77
Timestep 51, action = left, total score = 0
TrainState.MOVING
30 32
TrainState.DONE
92 102
Timestep 63, action = left, total score = 0
TrainState.DONE